<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/03_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# トピックモデリング (topic modeling)

* BoW (bag-of-words) の範囲で実現できる優れたEDA (exploratory data analysis)。


## 解説

### 使いみち
* テキストの集合から、多数の異なる話題を、それぞれの話題を端的に表す単語リストとして取り出せる。
* BoWの文書ベクトル（単語の出現頻度を要素とするベクトル）の次元圧縮には、使わない方がよい。
 * あくまでEDAの手法として使うのが吉。

### 入力データの形式
* 入力データは各文書における各単語の出現回数。
 * BoWとしてテキストをモデリングするので、**語順は考慮されない**。

### 代表的な手法: 潜在的ディリクレ配分法
* 英語ではLDA (latent Dirichlet allocation)。
* LDAはテキスト集合のモデリングに使えるベイズ的な確率モデル。
 * LDAの理屈については「統計モデリング2」で。
* 今回はsklearnの実装を使う。
 * https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
* gensimのLDAの実装はお勧めしない。
 * デフォルトの設定が間違っているため。

### LDAのモデル構成
* LDAは、テキスト集合から、$K$個のトピックを抽出する。
* 各トピックは、$W$個の語彙の上に定義された確率分布として得られる。
 * 各トピックについて、全語彙にわたって和をとると1になる数値の集まりが得られる。
 * $\phi_k = \{ \phi_{k,1}, \ldots, \phi_{k,W} \}$ s.t. $\sum_{w=1}^W \phi_{k,w} = 1$ for $k=1, \ldots, K$
* LDAを使うと、各テキストにおけるトピックの混合率も分かる。
 * 各テキストについて、全てのトピックにわたって和を求めると1になる数値の集まりが得られる。
 * $\theta_d = \{ \theta_{d,1}, \ldots, \theta_{d,K} \}$ s.t. $\sum_{k=1}^K \theta_{d,k} = 1$ for each document $d$
* 今回は、各トピックにおいて確率の高い単語を、ワードクラウドで可視化する。

## spaCy日本語モデルのインストール

In [ ]:
!python -m spacy download ja_core_news_sm

## データセットの準備
* liverdoorニュースコーパスを使う。

In [ ]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz

* 前回と同じ前処理。

In [ ]:
import re
import tarfile

tar_fname = "ldcc-20140209.tar.gz"

def read_title(f):
  next(f) # URL
  next(f) # タイムスタンプ
  title = next(f) # 3行目を返す：タイトル
  title = title.decode('utf-8')
  brackets_tail = re.compile('【[^】]*】$')
  brackets_head = re.compile('^【[^】]*】')
  return re.sub(brackets_head, "", re.sub(brackets_tail, "", title))[:-1]

corpus = []
with tarfile.open(tar_fname) as tf:
  for item in tf:
    if "LICENSE.txt" in item.name:
      continue
    if len(item.name.split('/')) < 3:
      continue
    if not item.name.endswith(".txt"):
      continue
    fname = item.name
    # 今回はクラス名は要らない
    #class_name = fname.split('/')[1]
    f = tf.extractfile(fname)
    title = read_title(f)
    corpus.append(title)

In [ ]:
len(corpus)

* 形態素解析し、活用語は原形に戻す。
* 今回は、名詞、固有名詞、動詞、形容詞、副詞のみを残す。

In [ ]:
import spacy
from tqdm import tqdm

nlp = spacy.load("ja_core_news_sm")

pos_list = ["NOUN", "PROPN", "VERB", "ADJ", "ADV"]

lemmatized = []
for text in tqdm(corpus):
  words = [token.lemma_ for token in nlp(text) if token.pos_ in pos_list]
  lemmatized.append(' '.join(words))

In [ ]:
lemmatized[:20]

In [ ]:
with open("lemmatized_livedoor_corpus.txt", "w") as f:
  for text in lemmatized:
    f.write(f"{text}\n")

## word cloudを作る練習
* livedoorニュースコーパス全体で一つのword cloudを作ってみる。

In [ ]:
import codecs
import matplotlib.pyplot as plt
from wordcloud import WordCloud

%config InlineBackend.figure_format = 'retina'

### テキストの準備
* 全てのテキストをつなげた長い文字列を作る。

In [ ]:
with open("lemmatized_livedoor_corpus.txt", "r") as f:
  lines = f.readlines()
long_text = ' '.join([line.strip() for line in lines])
long_text[:50]

### 単語のフィルタリング

* 単語を出現頻度の降順にソートする。

In [ ]:
from collections import Counter

word_freqs = Counter(long_text.split()).items()
sorted_word_freqs = sorted(word_freqs, key=lambda x: -x[1])

In [ ]:
print(sorted_word_freqs[:50])

* 適当な条件を設定してフィルタリングする。

In [ ]:
reduced_sorted_word_freqs = [
    (word, freq)
    for word, freq in sorted_word_freqs
    if freq < 320 and freq >= 5 and len(word) > 1
    ]
print(reduced_sorted_word_freqs[:10])
print(reduced_sorted_word_freqs[-10:])

### word cloudの描画

In [ ]:
wordcloud = WordCloud(
    font_path="/usr/share/fonts/truetype/fonts-japanese-mincho.ttf",
    background_color="white",
    width=1600,
    height=900,
    )
wordcloud.generate_from_frequencies(dict(reduced_sorted_word_freqs))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("word_cloud.png")

* 多数のテキストに対して、たった一つword cloudを作ったところで、何が分かるというのだろうか？

## LDAによるEDA

### データ行列の作成
* LDAの場合、単語の出現頻度をそのまま使って各文書をベクトル化する。

* 先ほど決めた単語群をLDAの語彙として使う。

In [ ]:
vocabulary = dict(reduced_sorted_word_freqs).keys()
len(vocabulary)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

with open("lemmatized_livedoor_corpus.txt", "r") as f:
  lines = f.readlines()
corpus = [line.strip() for line in lines]

# 英語の単語は小文字にしないようにする
vectorizer = CountVectorizer(lowercase=False, vocabulary=vocabulary)
X = vectorizer.fit_transform(corpus)

* 文書数と語彙サイズを変数にセット

In [ ]:
n_samples, n_features = X.shape

### LDAによるトピック抽出の実行
* 内部的には、変分推論で事後分布のパラメータを推定している。
 * `learning_method="online"`として、ミニバッチ式の繰り返し計算にすることを推奨。

* 抽出するトピックの個数は`n_components`で指定する。

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 20

lda = LatentDirichletAllocation(
    n_components=n_components, #要チューニング
    doc_topic_prior=0.05, #要チューニング
    topic_word_prior=0.01, #要チューニング
    learning_method="online",
    max_iter=20,
    batch_size=200,
    random_state=12345,
    evaluate_every=1,
    verbose=1, #最適化計算の進行状況をチェックする
    )

In [ ]:
lda.fit(X)

### 高確率語をワードクラウドで可視化

In [ ]:
wordcloud = WordCloud(
    font_path="/usr/share/fonts/truetype/fonts-japanese-mincho.ttf",
    background_color="white",
    width=1600,
    height=900,
    )

In [ ]:
n_cols = 4

fig, axes = plt.subplots(
    n_components // n_cols ,
    n_cols,
    figsize=(16, 16),
    sharex=True,
    sharey=True,
    )

for i, ax in enumerate(axes.flatten()):
  fig.add_subplot(ax)
  # キーが単語で値が重みの辞書を作っている
  wordcloud.generate_from_frequencies(
      dict(zip(vocabulary, lda.components_[i]))
      )
  plt.gca().imshow(wordcloud)
  plt.gca().set_title(f"Topic {i:02d}")
  plt.gca().axis('off')

plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()

## LDAのチューニング
* perplexityの値ができるだけ小さくなるように、チューニングする。
* 計算に時間がかかるからといって、`max_iter`を一桁にしないこと。
 * `max_iter`の値は十分に大きくすること。
 * perplexityの値があまり動かないところまで推定計算をちゃんと動かすため。

### チューニングすべきパラメータ
* `n_components`
 * 抽出するトピックの数。
 * 多すぎても、少なすぎても、分析が今ひとつになる。
 * 数千件のテキストなら、2桁のトピック数はおそらく必要。
* `doc_topic_prior`
 * ドキュメントごとのトピック確率分布の事前分布のパラメータ。
 * 詳細は「統計モデリング2」で。
 * 0.01, 0.02, 0.05, 0.1, 0.2, 0.5の6通りぐらいは試す。
* `topic_word_prior`
 * トピックごとの単語確率分布の事前分布のパラメータ。
 * 詳細は「統計モデリング2」で。
 * 0.01, 0.02, 0.05, 0.1, 0.2, 0.5の6通りぐらいは試す。

## pyLDAvisによる可視化
* チューニングが終わった後に使うと良い。

* インストール

In [ ]:
!pip install pyLDAvis

* おそらくランタイムの再起動が必要。

* LDAの学習をやり直す。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

with open("lemmatized_livedoor_corpus.txt", "r") as f:
  lines = f.readlines()
corpus = [line.strip() for line in lines]

vectorizer = CountVectorizer(max_df=0.5, min_df=5)
X = vectorizer.fit_transform(corpus)

n_components = 20

lda = LatentDirichletAllocation(
    n_components=n_components,
    doc_topic_prior=0.05,
    topic_word_prior=0.01,
    learning_method='online',
    max_iter=20,
    batch_size=200,
    random_state=12345,
    evaluate_every=1,
    verbose=1,
    )
lda.fit(X)

In [ ]:
import pyLDAvis

pyLDAvis.prepare(
  lda.components_,
  lda.transform(X),
  doc_lengths=X.sum(axis=1).getA1(),
  vocab=vectorizer.get_feature_names_out(),
  term_frequency=X.sum(axis=0).getA1(),
  #mds="tsne",
)